### Inference only

In [1]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch
import torch.nn.functional as F  # To use softmax

# Path to the saved model and tokenizer
model_path = './output/distilbert-sst-2'

# Load the model and tokenizer from the saved path
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)

# Set the model to evaluation mode (disable dropout, etc.)
model.eval()

def preprocess_text(text, tokenizer, max_length=256):
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    return encoding

def predict_sentiment(text, model, tokenizer):
    # Preprocess the input text
    encoding = preprocess_text(text, tokenizer)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits  # Get the raw prediction scores
        
        # Apply softmax to get probabilities
        probabilities = F.softmax(logits, dim=-1)  # Apply softmax along the last dimension
        predicted_class = torch.argmax(probabilities, dim=-1)  # Get the predicted class (0 for negative, 1 for positive)
    
    # Map the predicted class to sentiment label
    sentiment = 'POSITIVE' if predicted_class.item() == 1 else 'NEGATIVE'
    
    # Return sentiment and probabilities for each class
    positive_prob = probabilities[0][1].item()  # Probability for POSITIVE class
    negative_prob = probabilities[0][0].item()  # Probability for NEGATIVE class
    
    return sentiment, positive_prob, negative_prob

OSError: ./output/distilbert-sst-2 does not appear to have a file named config.json. Checkout 'https://huggingface.co/./output/distilbert-sst-2/tree/main' for available files.

In [ ]:
# Sample sentences (positive and negative examples)
sample_sentences = [
    "I absolutely love this movie! It was amazing.",
    "This movie was terrible, I hated every second of it.", 
    "while this movie is not intended for everyone, it is good for someone has no brain", 
    "let's watch it only when it is free to watch, i will not pay for it",
    'A worthy contender for the Animated film of 2024', 
    'No plot at all. But if you are looking for a good laugh. You will not find that either.'
]

# Perform inference on each sample sentence
for sentence in sample_sentences:
    sentiment, positive_prob, negative_prob = predict_sentiment(sentence, model, tokenizer)
    print(f"Sentence: {sentence}\nPredicted Sentiment: {sentiment}")
    print(f"POSITIVE: {positive_prob:.4f}, NEGATIVE: {negative_prob:.4f}\n")

Sentence: I absolutely love this movie! It was amazing.
Predicted Sentiment: POSITIVE
POSITIVE: 0.9998, NEGATIVE: 0.0002

Sentence: This movie was terrible, I hated every second of it.
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0004, NEGATIVE: 0.9996

Sentence: while this movie is not intended for everyone, it is good for someone has no brain
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0683, NEGATIVE: 0.9317

Sentence: let's watch it only when it is free to watch, i will not pay for it
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0671, NEGATIVE: 0.9329

Sentence: A worthy contender for the Animated film of 2024
Predicted Sentiment: POSITIVE
POSITIVE: 0.9996, NEGATIVE: 0.0004

Sentence: No plot at all. But if you are looking for a good laugh. You will not find that either.
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0017, NEGATIVE: 0.9983

